In [1]:
#install praw

#%pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
#set up
import requests
import os
import pandas as pd
import praw
import csv

In [29]:
#authorize praw
reddit = praw.Reddit(
    client_id= "L9bDp7Cr1Uianzb7xtzeXw",
    client_secret= "WCJSgyUcmO9hyu-ItBO1zAKiIX-sBQ" , 
    user_agent= "python:project-script:v1.0 (by u/Username10907)" )

In [33]:
#test
test_subreddit = reddit.subreddit("redditdev")
print(test_subreddit.display_name)

book_subreddit = reddit.subreddit("books")
print(book_subreddit.title)
print(book_subreddit.description)

redditdev
So many books, so little time
###### [](#place announcements below)

* New Release: [City of Night Birds by Juhea Kim](https://www.goodreads.com/search?&query=9780063394759)
* Check out the [Weekly Recommendation Thread](https://redd.it/1gx59uk)
* Join in the [Weekly "What Are You Reading?" Thread!](https://redd.it/1gzgayd)


## [- Subreddit Rules -](/r/books/wiki/rules)[- Message the mods -](http://goo.gl/HXpfgH)[Related Subs](/r/books/wiki/relatedsubreddits)[AMA Info](/r/Books/wiki/amarules)[The FAQ](/r/books/wiki/faq) [The Wiki](/r/books/wiki/index)

This is a moderated subreddit. It is our intent and purpose to foster and encourage in-depth discussion about all things related to books, authors, genres or publishing in a safe, supportive environment. If you're looking for help with a personal book recommendation, consult our [Suggested Reading](/r/books/wiki/suggested) page or ask in: /r/suggestmeabook

# Quick Rules:

1. **Discussion is the goal**  
Do not post shallow co

In [35]:
#pulling from r/washingtondc

my_subreddit = "washingtondc"
submission_headers = ['author', 'created_utc', 'id', 
                      'is_original_content', 'is_self', 
                      'link_flair_text', 'locked', 'name', 
                      'num_comments', 'over_18', 'permalink', 
                      'score', 'selftext', 'spoiler', 'stickied', 
                      'subreddit', 'title', 'upvote_ratio', 'url']

#Note: 'a' opens the file in append mode to avoid overwriting data
with open("reddit_test_submission_db.csv", 'a', 
          encoding="utf-8", newline='') as f_object:
    newposts = reddit.subreddit(my_subreddit).new(limit=None)
    for post in newposts:
    #Below are all the fields we'll request from PRAW for each post
        data = {'author': post.author, 'created_utc': 
                post.created_utc, 'id': post.id, 
                'is_original_content': post.is_original_content, 
                'is_self': post.is_self, 'link_flair_text': 
                post.link_flair_text, 'locked': post.locked, 
                'name': post.name, 'num_comments': 
                post.num_comments, 'over_18': post.over_18, 
                'permalink': post.permalink, 'score': post.score, 
                'selftext': post.selftext, 'spoiler': post.spoiler, 
                'stickied': post.stickied, 'subreddit': 
                post.subreddit, 'title': post.title, 
                'upvote_ratio': post.upvote_ratio, 'url': post.url}
        dictwriter_object = csv.DictWriter(
            f_object, fieldnames=submission_headers)
        dictwriter_object.writerow(data)
    f_object.close()
    
#Code below will delete duplicates on successive pulls
post_db = pd.read_csv("reddit_test_submission_db.csv", 
                      names=submission_headers, header=0)
post_db.drop_duplicates(subset="permalink", 
                        keep="last", inplace=True)
post_db.to_csv("reddit_test_submission_db.csv", 
               index=False, chunksize=1000)

In [ ]:
#This block creates list of submissions for which we want comments
comment_headers = ['author', 'body', 'created_utc', 
                   'distinguished', 'edited', 'id', 
                   'is_submitter', 'link_id', 'parent_id', 
                   'permalink', 'saved', 'score', 'stickied', 
                   'submission', 'subreddit', 'subreddit_id']
with open('reddit_test_comment_db.csv', 'a') as comment_file:
    comments_db = pd.read_csv('reddit_test_comment_db.csv', 
                              usecols=["submission"], 
                              names=comment_headers)
    comment_file.close()
submission_db = pd.read_csv("reddit_test_submission_db.csv", 
                            usecols=["created_utc", "id"])
#Filter down to submissions for which we don't yet have comments
comments_set = set(comments_db["submission"])
#259,200 is three days worth of seconds
try:
    time_cutoff = max([created_utc for post_id, created_utc in 
         zip(submission_db.id, submission_db.created_utc) 
         if post_id in comments_set]) - 259200
except:
    time_cutoff = submission_db["created_utc"].min()
submissions_to_pull = submission_db.loc[submission_db
                                        ["created_utc"] >= 
                                        time_cutoff, "id"]
#This block pulls all comments for the list of submissions we have identified
with open("reddit_test_comment_DB.csv", 'a', 
          encoding="utf-8", newline='') as f_object:
    for row in submissions_to_pull:
        submission = reddit.submission(id=row)
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            data = {'author': comment.author, 'body': 
                    comment.body, 'created_utc': 
                    comment.created_utc, 'distinguished': 
                    comment.distinguished, 'edited': 
                    comment.edited, 'id': comment.id, 
                    'is_submitter': comment.is_submitter, 
                    'link_id': comment.link_id, 'parent_id': 
                    comment.parent_id, 'permalink': 
                    comment.permalink, 'saved': comment.saved, 
                    'score': comment.score, 'stickied': 
                    comment.stickied, 'submission': 
                    comment.submission, 'subreddit': 
                    comment.subreddit, 
                    'subreddit_id': comment.subreddit_id}
            dictwriter_object = csv.DictWriter(f_object, 
                                               fieldnames=
                                               comment_headers)
            dictwriter_object.writerow(data)
    f_object.close()
    
#Now drop duplicate rows
comment_db = pd.read_csv("reddit_test_comment_DB.csv", 
                         names=comment_headers, header=0)
#First drop duplicates that are not edited, keep the last pull
comment_db.drop_duplicates(subset=["permalink", "body", "id"],
                           keep = "last", inplace=True)
#Then drop duplicates that have been edited, keep the first pull
comment_db.drop_duplicates(subset="permalink", 
                           keep = "first", inplace=True)
comment_db.to_csv("reddit_test_comment_DB.csv", 
                  index=False, chunksize=1000)